<a href="https://colab.research.google.com/github/taylorfrancisco/Example-Work/blob/main/Francisco_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from __future__ import division
from statsmodels.compat import lzip
import scipy as sp


In [ ]:
!pip install linearmodels
from linearmodels.iv import IV2SLS

In [ ]:
#create dataframe of variables of interest
d = pd.read_csv(('GSS_Cum.csv'),
usecols= ['workblks', 'health', 'realinc', 'year', 'cohort', 'race', 'racwork'])


In [ ]:
#recode to make variables more interpretable
d['workblks'] = 7 - d.workblks
d['health'] = 4 - d.health
d['inc10k'] = d.realinc/10000
#remove all non-values
data= d.dropna()


# Variables

 I am interested in exploring whether being biased or racist impacts overall health. The logic being that if a person has a severe bias against a group of people, that will lead to higher cortisol levels whenever they are exposed to somone who is a part of that group, due to an increase in fear or frustration that they associate with that group. Having frequently high cortisol is deleterious to health. I am curious to see if this hypothesized increase in cortisol would produce a causal relationship between a racism indicator and perceived health. If my assumption holds true, my independent variable of perceived level of how hardworking black people are will be predictive of worse overall health.

### Independent Variable

My independent variable for this exercise is 'Workblks'. This variable has respondents ranking how hard working they believe black people to be from hard working (1) to lazy (7). I am using this a proxy for bias against black people which may indicate a level of racism that may lead to increase stress/cortisol levels as noted above. I recoded this variable to be increasing in perceived hard working-nes to be more intuitive to interpret, so higher numbers indicate more positive views.

### Dependent Variable

My dependent variable for this excercise is 'health' which asks respondents to rank from execellent to poor what would say their own health is. I again recoded this variable to go from poor to excellent so that higher numbers indicate better health, making the variable more easily interpretable.




### Controls


For some basic controls I included the year of the survey response, the cohort of the respondent (how old they are), and their race (white- 1, black - 2, or other -3). I also included real income which is the income in real dollars which I recoded to be in icrements of 10 thousand dollars. The assumptions are that richer individuals will have higher levels of health and that race will also impact how the person views black people. Age also would presumably impact health levels and also older people may be more likely to be more prejudiced.


# OLS

In [ ]:
# Fit the linear regression model
lm1 = smf.ols('health ~ workblks + inc10k + year + cohort + C(race)', data = data).fit()

# Print the model summary
print(lm1.summary())

According to this OLS, for every increase in positive view of black people, people on average increase their health by 0.0234 net of all other variables. This is statistically significant and confirms my assumption that those with less bias have better health (ie those with more prejudice have worse health).

For every 10,000 dollar increase in income, people on average have a 0.0322 increase in health, net of all other variables. This is also statistically significant and confirms my assumption that more wealth leads to better health.

 Black people compared to white people have 0.1117 points lower health, net of all other variables. This is also statistically significant and makes sense as structural determinents of health usually leads to black people having worse health outcomes.

 People who fall into the 'other' race category compared to whites have on average 0.0820 points lower health, net of all variables. This is not quite statistically significant (p-value of 0.067) but again follows similar logic to black people as non-white individuals tend to face worse structural determinents of health leading to worse health.

 Laslty, for every year increase in cohort, people on average have a 0.0051 increase in health, net of all other variales,which is statistically significant. This does not confirm previous assumptions of older individuals having worse health.

### Critique of OLS

 While this OLS does support my hypothesis, it may not be the best model for my question as how people perceive black people might be endogenous to health. There may be an omitted variable that is driving both health and perception of black people that is obscuring the direct relationship between racism and health. For example, it is possible that some people are just more generally positive in life and thus have a more optimisitc view of their health and kinder perceptions of people in general. This type of omitted variable is difficult to measure and thus cannot be controlled for. Instead, an instrumental variable could be used to circumvent this variable by using a proxy of such for perception of black people that is not impacted by general optimism. Using a proxy that is more or less random that would still be highly correlated with perception of black people would allow for stronger causal conclusions to be made.

# Instrumental Variable

## Instrument

For my isntrumental variable I am going to use 'Racwork', which is the racial makeup of workplace. This variable ranges from 1 (all white workplace) to 5 (all blakc workplace). A value of 6 indicates that the person works alone, which for the purpose of this excercise I will exclude.

I chose this variable as it is somewhat random, a person doesn't necessarily choose the racial makeup of their workplace. It also should not be directly related to health outcomes, it shouldn't in theory matter for your health who you work with. Additionally, it is also not related to the potential omitted variable of being a more positive individual (ie z is uncorrelated with u).

This instrumental variable though could be related to the amount of prejudice a person has, as the more exposure a person has to people in a group, the less prejudiced they may become.


In [ ]:

#drop value 6 which does not apply to analysis
data = data.drop(data[data['racwork'] == 6].index)

print(data['racwork'].max())


# IV Model

In [ ]:
from linearmodels.iv.model import IV2SLS

iv1 = IV2SLS.from_formula('health ~ inc10k + year + cohort + C(race) + [workblks  ~ racwork]', data=data).fit()
iv1



According to this IV model, for every increase in perception of black people, people on average decrease their overall health scores by 5.2222, net of all other variables. This is no longer statistically significant in this model and is in the reverse direction as the OLS model. This is likely due to racwork not being a valid instrument, which will be discussed more in the diagnostic section. The larger standard error of 21.608 in this model compared to 0.010 of the OLS mdoel though is expected, as IVs are less efficient and increases standard errors and p-values.

Conceptually, racwork being a poor IV becomes clear on reflection. If my assumption is that cortisol increases when a person is exposed to someone who is a part of the group they are biased against, exposure in the workplace would thus have the effect of raising cortisol and worsening health (assuming they are indeed racist to begin with). Since the composition of the workplace is later shown to not be strongly correlated with my indicator of prejudice, it is only adding random noise to my model, producing hard to interpret coefficients as opposed to causal relationships.

The control variables in this model were also highly impacted by the inefficient nature of IV models as none of these coefficients are statistically significant anymore. For income, the direction and magnitude was relatively strong however, with a change from 0.0322 in the OLS model to a coefficient of 0.1146 in the IV model. Similarly for the cohort coefficient, it is fairly similar between the two models with OLS being 0.0051 and the IV model being 0.0463. This may indicate that the relationship between age/income and health is rather robust.

The coefficient for the race controls however were wildly impacted by the IV model, going from modest levels of -0.1117  for black individuals and -0.0820 for 'other' individuals to -51.949 and -56.205 respectively. This is another indicator that the instrumental variable of racwork is introducing a lot of likely invalid noise into the model.


# Diagnostics

In [ ]:
#determine if racwork is highly correlated with independent variable

lm_strong = smf.ols('workblks  ~ racwork', data = data).fit()

# Print the model summary
print(lm_strong.summary())

The first verifiable assumption of IV is that your instrument is strongly correlated with your independent variable, which is indicated by a high r-squared when regressing the IV on the independent variable. For my chosen IV, this assumption is violated with a very weak r-squared of 0.006. This indicated that there is very little relevance to my variable and it should not be used as an instrumental variable for my variables of interest.

In [ ]:
#determine if racwork is highly correlated with dependent variable on its own

lm_pred = smf.ols('health  ~ racwork', data = data).fit()

# Print the model summary
print(lm_pred.summary())

Another assumption of the IV model is that the instrument is not directly correlated with the dependent variable, as a quick check I regressed my instrument on the dependent variable to see if there is a relationship between them without my independent variable as an intermediate. This showed that for every increase in diversity in the workplace, a person on average has a decrease in health by -0.0552, which is statistically significant. As there seems to be a relationship between these two variables by themselves, it seems that this assumption is violated and again my chosen instrument is not valid.

In [ ]:
# Tests for IV specification
# Wu-Hausman test of exogeneity
print(iv1.wu_hausman())



Again, these diagnostic tests indicate that my chosen instrumental variable was a weak instrument, with an F statistic from the first stage of 0.1124, which is much much smaller than the general rule of thumb of 10 or higher.

However, the Wu-Hausman test had a t-statistics of 5.6436, which was statistically significant, which does indicate that my independent variable was endogenous to health. This suggests that an instrumental variable is needed to improve the first OLS model.


# Conclusion

Overall, my chosen instrumental variable was weak and was not a valid choice to model the relationship between racism and health. However, the Wu-Hausman test indicated that there was an omitted variable that should be addressed, meaning my OLS is also not a good model to describe the relationship between my variables of interest. As such, there is no concrete conclusion from these models and a new instrumntal variable should be found to try to get a clearer picture of how racism and health are related. While the OLS did support my initial hypothesis, it is also possible that my proxy for racism (how individuals view the hard-working nature of black people) could also not be a valid proxy. More exploration should be done finding other potential indicators of racism and also crucially other instrumental variables that are strong are necessary to really answer my question of interest. There does appear to be a strong relationship between income and age on health and these variables should definitely be included as controls in future models.